#mainVacationPy

Maria Barrera -- 02/07/2021

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# data file -- created from mainWeatherPy
city_path = "output_data/cities_file.csv"
city_path

'output_data/cities_file.csv'

In [3]:
# Read the city data 
city_data = pd.read_csv(city_path)
city_data

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,9505,carnarvon,0,AU,1612811945,100,-24.8667,113.6333,298.15,8.23
1,2011314,port alfred,79,ZA,1612812024,88,-33.5906,26.8910,292.59,0.89
2,8303,ushuaia,0,AR,1612811989,55,-54.8000,-68.3000,290.15,7.20
3,5832,saint george,1,US,1612811821,27,37.1041,-113.5841,287.04,1.54
4,8926,nikolskoye,0,RU,1612812012,84,59.7035,30.7861,255.93,1.00
...,...,...,...,...,...,...,...,...,...,...
302,8587,san cristobal,40,VE,1612812806,48,7.7669,-72.2250,305.15,5.14
303,8368,itaituba,40,BR,1612812568,66,-4.2761,-55.9836,304.15,2.06
304,2434,atar,0,MR,1612812807,12,20.5169,-13.0499,300.15,3.09
305,8675,pucallpa,40,PE,1612812572,58,-8.3791,-74.5539,305.15,3.09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Convert humidity as a list
humidity_list = city_data['Humidity']
humidity_list

0      100
1       88
2       55
3       27
4       84
      ... 
302     48
303     66
304     12
305     58
306    100
Name: Humidity, Length: 307, dtype: int64

In [5]:
# Create a map using city coordinates to set markers
marker_locations = city_data[['Lat', 'Lng']]

# Create a marker_layer using humidity list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Humidity: {humidity}" for humidity in humidity_list])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#for reference only
city_data.columns

Index(['City ID', 'City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Lat',
       'Lng', 'Max Temp', 'Wind Speed'],
      dtype='object')

In [7]:
#  temperature conversion & column update

new_df = city_data

# for testing only ---------------
# maxtemp = 294.26
# temp_f = (maxtemp - 273.15) * 9/5 +32
# temp_f
# --------------------------------

for ind in new_df.index: 
    maxtemp = new_df['Max Temp'][ind]
    temp_f = (maxtemp - 273.15) * 9/5 +32
    new_df['Max Temp'][ind] = temp_f
    
    # for testing only
    #print(temp_f)
    #print(new_df['Max Temp'][ind], ind) 
    
new_df.head()

<ipython-input-7-51d7cc9f05bd>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Max Temp'][ind] = temp_f


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,9505,carnarvon,0,AU,1612811945,100,-24.8667,113.6333,77.000,8.23
1,2011314,port alfred,79,ZA,1612812024,88,-33.5906,26.8910,66.992,0.89
2,8303,ushuaia,0,AR,1612811989,55,-54.8000,-68.3000,62.600,7.20
3,5832,saint george,1,US,1612811821,27,37.1041,-113.5841,57.002,1.54
4,8926,nikolskoye,0,RU,1612812012,84,59.7035,30.7861,1.004,1.00


In [8]:
#vacation selection criteria
#temperature 70-80 degrees
#wind speed <10
#cloudiness = 0

subset_df = new_df[(new_df["Max Temp"] >= 70)]
subset_df = subset_df[subset_df["Max Temp"] <= 80]
subset_df = subset_df[subset_df["Wind Speed"] < 10]
subset_df = subset_df[subset_df["Cloudiness"] == 0]
subset_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,9505,carnarvon,0,AU,1612811945,100,-24.8667,113.6333,77.000,8.23
45,7562,salalah,0,OM,1612812056,73,17.0151,54.0924,73.400,1.03
101,8691,pisco,0,PE,1612812644,73,-13.7000,-76.2167,77.000,3.60
123,2030260,viedma,0,AR,1612812025,38,-40.8135,-62.9967,75.992,1.34
146,8480,calama,0,CL,1612812432,29,-22.4667,-68.9333,75.200,9.26
246,9233,varca,0,IN,1612812507,69,15.2167,73.9167,77.000,3.60


In [9]:
# Drop any rows will null values
subset_df = subset_df.dropna(how="any")
subset_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,9505,carnarvon,0,AU,1612811945,100,-24.8667,113.6333,77.000,8.23
45,7562,salalah,0,OM,1612812056,73,17.0151,54.0924,73.400,1.03
101,8691,pisco,0,PE,1612812644,73,-13.7000,-76.2167,77.000,3.60
123,2030260,viedma,0,AR,1612812025,38,-40.8135,-62.9967,75.992,1.34
146,8480,calama,0,CL,1612812432,29,-22.4667,-68.9333,75.200,9.26
246,9233,varca,0,IN,1612812507,69,15.2167,73.9167,77.000,3.60


In [10]:
# create 2nd map - selected vacation cities
# Convert humidity as a list
humidity_list2 = subset_df['Humidity']
humidity_list2

# Create a map using city coordinates to set markers
marker_locations = subset_df[['Lat', 'Lng']]

# Create a marker_layer using humidity list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Humidity: {humidity}" for humidity in humidity_list2])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# set up hotel df to hold information  
hotel_df = pd.DataFrame()

hotel_df['Hotel Name'] = ""
hotel_df

,Hotel Name


#### Need to get hotel info

#code to store lat /lng -- need to replace 'subset_df' with 'hotel_df'
lat_list = subset_df['Lat']
lat_list

for item in lat_list:
    hotel_df['Lat'] = subset_df['Lat']
hotel_df

lng_list = subset_df['Lng']
lng_list

for item in lng_list:
    hotel_df['Lng'] = subset_df['Lng']   
hotel_df

##### NOTE: Do not change any of the code in this cell

###### Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
##### Store the DataFrame Row
##### NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

#### # Add marker layer ontop of heat map


#### # Display figure


#### Plot Heatmap with Markers
fig = gmaps.figure()

####  Add markers for hotels
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#### Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

#### Add layers
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


#####  Display figure
fig